<a href="https://colab.research.google.com/github/kavyajeetbora/civil_engineering/blob/master/pile_lateral_capacity/Lateral_Capacity_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reading file into raw representation using python default repr (representation) function

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import ipywidgets as widgets

In [2]:
df = pd.read_excel("/content/drive/My Drive/Colab Notebooks/Projects/office work/Pile Lateral Capacity/stiffness.xlsx")
df = df.drop(index=4)
df.head()

,Pier no.,P0/13,P0/15,P0/19,P0/21,P0/23,P0/25,ST1/0,ST1/C,ST1/E,...,BH-P3/06,BH-P3/08,P3-02,P3/10,P3/12,P3/14,P3/16,P3/18,P3/20,P3/22
0,Reference Borehole,BH-P0/13,BH-P0/15,BH-P0/17,BH-P0/19,BH-P0/21,BH-P0/23,BH-ST1/1,BH-ST1/C,BH-ST1/E,...,BH-P3/06,BH-P3/08,BH-P3-02,BH-P3/10,BH-P3/12,BH-P3/14,BH-P3/16,BH-P3/18,BH-P3/20,BH-P3/22
1,"Ground Level m, MSL",617.29,614.7,611.57,610.45,608.66,608.21,608.57,608.93,608.93,...,611.62,612.46,608.89,611.71,609.8,608.82,609.44,610.88,612.38,613.86
2,Top of Pile Level (TPL),612.79,612.7,609.57,608.45,606.66,606.21,606.57,606.93,606.93,...,609.62,610.46,606.89,609.71,607.8,606.82,607.44,608.88,610.38,611.86
3,Lateral Stiffness (kN/m),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TPL-1.5m,1710.0,386783.0,42000.0,386783.0,386783.0,42000.0,386783.0,3552401.0,4500.0,...,386783.0,8938387.0,42000.0,8938387.0,386783.0,42000.0,386783.0,386783.0,42000.0,42000.0


## 1. Inputs

### 1.1 Widgets

In [3]:
style = {'description_width': 'initial'}

freelength = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Free Length of the pile:',
    style = style,
    disabled=False
)
pile_head_cond = widgets.RadioButtons(
    options=['Free', 'Fixed'],
    value='Free',
    description='Pile Head Conditions:',
    style = style,
    disabled=False
)

### 1.2 User Inputs

In [4]:
display(freelength, pile_head_cond)

BoundedFloatText(value=0.5, description='Free Length of the pile:', max=10.0, step=0.1, style=DescriptionStyle…

RadioButtons(description='Pile Head Conditions:', options=('Free', 'Fixed'), style=DescriptionStyle(descriptio…

In [5]:
free_length = freelength.get_interact_value()
fixed_head = False if pile_head_cond.get_interact_value() == "Free" else True

## 2. Processing Data

In [6]:
piers = df.columns
piers

Index(['Pier no.', 'P0/13', 'P0/15', 'P0/19', 'P0/21', 'P0/23', 'P0/25',
       'ST1/0', 'ST1/C', 'ST1/E', 'ST1/G', 'P1/01', 'P1/03', 'P1/06', 'P1/08',
       'P1/10', 'P1/12', 'P1/14', 'P1/16', 'P1/18', 'P1/20', 'BH-P1/24',
       'BH-P1/26', 'ST2/A', 'ST2/C', 'ST2/E', 'ST2/G', 'ST2/J', 'ST3/A',
       'ST3/B', 'ST3/C', 'ST3/D', 'ST3/E', 'ST3/F', 'ST3/G', 'ST3/H',
       'BH-P2/02', 'BH-P2/04', 'BH-P2/06', 'BH-P2/08', 'BH-P2/10', 'P2/12',
       'P2/14', 'P2/16', 'P2/18', 'P2/20', 'P2/22', 'BH-P3/04', 'BH-P3/06',
       'BH-P3/08', 'P3-02', 'P3/10', 'P3/12', 'P3/14', 'P3/16', 'P3/18',
       'P3/20', 'P3/22'],
      dtype='object')

In [7]:
node_nos = {}
start_node = 101
location = 0
for pier in piers[1:]:
    node_nos[pier] = (start_node, location, df.loc[5:,pier].dropna().astype(int).values)
    start_node += 100
    location += 10

node_no, location, spring_vals = node_nos['P0/13']
node_no + len(spring_vals)

111

In [9]:
joint_coordinates = ""
member_indices = ""
member_prop = ""
supports = ""
loads = ""
top_nodes = []

for pier in piers[1:]:
    node_no, location, spring_vals = node_nos[pier]
    if free_length > 0.0:
        joint_coordinates += "%d %d 0\n" % (node_no-1, location)
        member_indices += "%d %d %d\n" % (node_no-1, node_no-1, node_no) # 100 100 101 
        member_prop += "%d TO %d PRIS YD 1.0\n" % (node_no-1, node_no) #1 TO 8 PRIS YD 1.0
        if fixed_head:
            supports += "%d FIXED BUT FX FZ\n" % (node_no-1)
        top_node = node_no - 1
    else:
        top_node = node_no
    top_nodes.append(top_node)

    last_cors = 1 + len(spring_vals)
    last_node = node_no + len(spring_vals)
    
    joint_coordinates += "%d %d %.1f 0 %d %d -%.1f 0\n" % (node_no, location, -free_length, last_node, location, last_cors+free_length) # 101 0 0 0 110 0 -10 0
    member_indices += "%d %d %d %d\n" % (node_no, node_no, node_no+1, last_node-1) # 101    101    102    109
    member_prop += "%d TO %d PRIS YD 1.0\n" % (node_no, last_node-1) #1 TO 8 PRIS YD 1.0
    
    for i, spr_val in enumerate(spring_vals):
        if i == (len(spring_vals) -1):
            supports += "%d FIXED BUT MX MY MZ KFX %d KFZ %d;\n" % (node_no+i,spr_val,spr_val) # 1 FIXED BUT FY MX MY MZ KFX 1710 KFZ 1710;
        elif i==0 and free_length==0.0 and fixed_head:
            supports += "%d FIXED BUT FX FZ\n" % (node_no+i)
        else:
            supports += "%d FIXED BUT FY MX MY MZ KFX %d KFZ %d;\n" % (node_no+i,spr_val,spr_val) # 1 FIXED BUT FY MX MY MZ KFX 1710 KFZ 1710;
    supports += "\n"

    loads += "%d FX 100\n" % (top_node) # 1 FX 100

In [10]:
print("\n".join(joint_coordinates.split("\n")[:5]))
print("-"*30)
print("\n".join(member_indices.split("\n")[:5]))
print("-"*30)
print("\n".join(member_prop.split("\n")[:5]))
print("-"*30)
print("\n".join(supports.split("\n")[:11]))
print("-"*30)
print("\n".join(loads.split("\n")[:5]))
print("-"*30)

100 0 0
101 0 -0.5 0 111 0 -11.5 0
200 10 0
201 10 -0.5 0 210 10 -10.5 0
300 20 0
------------------------------
100 100 101
101 101 102 110
200 200 201
201 201 202 209
300 300 301
------------------------------
100 TO 101 PRIS YD 1.0
101 TO 110 PRIS YD 1.0
200 TO 201 PRIS YD 1.0
201 TO 209 PRIS YD 1.0
300 TO 301 PRIS YD 1.0
------------------------------
101 FIXED BUT FY MX MY MZ KFX 1710 KFZ 1710;
102 FIXED BUT FY MX MY MZ KFX 2430 KFZ 2430;
103 FIXED BUT FY MX MY MZ KFX 2430 KFZ 2430;
104 FIXED BUT FY MX MY MZ KFX 386783 KFZ 386783;
105 FIXED BUT FY MX MY MZ KFX 386783 KFZ 386783;
106 FIXED BUT FY MX MY MZ KFX 386783 KFZ 386783;
107 FIXED BUT FY MX MY MZ KFX 386783 KFZ 386783;
108 FIXED BUT FY MX MY MZ KFX 1502707 KFZ 1502707;
109 FIXED BUT FY MX MY MZ KFX 1502707 KFZ 1502707;
110 FIXED BUT MX MY MZ KFX 10 KFZ 10;

------------------------------
100 FX 100
200 FX 100
300 FX 100
400 FX 100
500 FX 100
------------------------------


**Outline**
1. define each module separately
2. then concatenate and export to excel or .std file for staad input

## Compiling the STAAD input

In [11]:
now = datetime.now().strftime("%d-%b-%y")

staad_format = '''STAAD SPACE
START JOB INFORMATION
ENGINEER DATE {0}
END JOB INFORMATION
INPUT WIDTH 79
UNIT METER KN
JOINT COORDINATES
{1}
MEMBER INCIDENCES
{2}
DEFINE MATERIAL START
ISOTROPIC CONCRETE
E 3e+007
POISSON 0.17
DENSITY 25
ALPHA 1e-005
DAMP 0.05
TYPE CONCRETE
STRENGTH FCU 36000
END DEFINE MATERIAL
MEMBER PROPERTY AMERICAN
{3}
CONSTANTS
MATERIAL CONCRETE ALL
SUPPORTS
{4}

LOAD 1 LOADTYPE None  TITLE FX
JOINT LOAD
{5}
PERFORM ANALYSIS
FINISH
'''

staad_input = staad_format.format(now,joint_coordinates, member_indices, member_prop, supports, loads)
print(staad_input)

STAAD SPACE
START JOB INFORMATION
ENGINEER DATE 11-Mar-22
END JOB INFORMATION
INPUT WIDTH 79
UNIT METER KN
JOINT COORDINATES
100 0 0
101 0 -0.5 0 111 0 -11.5 0
200 10 0
201 10 -0.5 0 210 10 -10.5 0
300 20 0
301 20 -0.5 0 307 20 -7.5 0
400 30 0
401 30 -0.5 0 407 30 -7.5 0
500 40 0
501 40 -0.5 0 508 40 -8.5 0
600 50 0
601 50 -0.5 0 610 50 -10.5 0
700 60 0
701 60 -0.5 0 710 60 -10.5 0
800 70 0
801 70 -0.5 0 810 70 -10.5 0
900 80 0
901 80 -0.5 0 910 80 -10.5 0
1000 90 0
1001 90 -0.5 0 1007 90 -7.5 0
1100 100 0
1101 100 -0.5 0 1107 100 -7.5 0
1200 110 0
1201 110 -0.5 0 1205 110 -5.5 0
1300 120 0
1301 120 -0.5 0 1307 120 -7.5 0
1400 130 0
1401 130 -0.5 0 1406 130 -6.5 0
1500 140 0
1501 140 -0.5 0 1508 140 -8.5 0
1600 150 0
1601 150 -0.5 0 1608 150 -8.5 0
1700 160 0
1701 160 -0.5 0 1707 160 -7.5 0
1800 170 0
1801 170 -0.5 0 1810 170 -10.5 0
1900 180 0
1901 180 -0.5 0 1908 180 -8.5 0
2000 190 0
2001 190 -0.5 0 2007 190 -7.5 0
2100 200 0
2101 200 -0.5 0 2109 200 -9.5 0
2200 210 0
2201 210 -0.5 

## Export the file

In [ ]:
with open("P1 to P3.std","w") as std_file:
    std_file.write(staad_input)

In [ ]:
all_nodes = []
for pier in piers[1:]:
    if free_length > 0.0:
        top_node = node_nos[pier][0]-1
    else:
        top_node = node_nos[pier][0]

    all_nodes.append((pier,str(top_node)))

In [ ]:
node_df = pd.DataFrame(all_nodes, columns = ['Pier No.', 'Staad Node'])
node_df.head()

,Pier No.,Staad Node
0,P0/13,100
1,P0/15,200
2,P0/19,300
3,P0/21,400
4,P0/23,500


In [ ]:
node_df.to_excel("nodes.xlsx", index=False)